# Instalando o pymongo

In [1]:
!pip install pymongo --break-system-packages

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 6.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 17.8 MB/s eta 0:00:00


# Teste básico de conexão e inserção de dados

In [115]:
from pymongo import MongoClient

client = MongoClient("mongodb://meu-mongo:27017/")

In [116]:
# teste basico de criação de coleção e inserção de documento
db = client.test_database
collection = db.test_collection

result = collection.insert_one({"nome": "carlos"})
print(result.acknowledged)

True


# Inserindo os dados no mongo com o pymongo

In [117]:
import csv

In [118]:
db = client.ufrn
collection_discente = db.discentes

with open('discentes-2024.csv', 'r', newline='') as f:
    data = csv.DictReader(f, delimiter=';')
    for linha in data:
        collection_discente.insert_one(linha)   

# Entrando no mongoDB com o spark

In [120]:
from pyspark.sql import SparkSession 
from pymongo import *

spark = SparkSession.builder \
    .appName("MongoDB") \
    .config("spark.mongodb.input.uri", "mongodb://meu-mongo:27017/ufrn.discentes") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:10.4.0") \
    .getOrCreate()

# Realizando transformaçãoes com os dataframes

In [121]:
df = spark.read.format("mongodb") \
    .option("connection.uri", "mongodb://meu-mongo:27017") \
    .option("database", "ufrn") \
    .option("collection", "discentes").load()

In [122]:
df.columns

['_id',
 'ano_ingresso',
 'forma_ingresso',
 'id_curso',
 'id_unidade',
 'id_unidade_gestora',
 'matricula',
 'modalidade_educacao',
 'nivel_ensino',
 'nome_curso',
 'nome_discente',
 'nome_unidade',
 'nome_unidade_gestora',
 'periodo_ingresso',
 'sexo',
 'sigla_nivel_ensino',
 'status',
 'tipo_discente']

In [123]:
from pyspark.sql.functions import sum, desc
df.select('nome_curso').show(100, truncate=False)

[Stage 163:>                                                        (0 + 1) / 1]

+----------------------------------------------------------------------------------+
|nome_curso                                                                        |
+----------------------------------------------------------------------------------+
|MESTRADO PROFISSIONAL EM ENERGIA ELÉTRICA                                         |
|                                                                                  |
|CURSO DE PORTUGUÊS COMO LÍNGUA DE ACOLHIMENTO                                     |
|MESTRADO EM FONOAUDIOLOGIA                                                        |
|TECNOLOGIA DA INFORMAÇÃO                                                          |
|DIREITO                                                                           |
|ARQUITETURA E URBANISMO                                                           |
|CIÊNCIAS ECONÔMICAS                                                               |
|CURSO DE LÍNGUA INGLESA  BÁSICO                                

In [124]:
# Listar todos os alunos que ingressaram por meio do SiSU;

alunos_sisu = df.filter(df.forma_ingresso == "SiSU")
alunos_sisu_count = df.filter(df.forma_ingresso == "SiSU").count()
alunos_sisu.show()
print(f'quantidade de alunos que ingressaram pelo sisu = {alunos_sisu_count}, alunos totais = {df.count()}')

+--------------------+------------+--------------+---------+----------+------------------+-----------+-------------------+------------+--------------------+--------------------+--------------------+--------------------+----------------+----+------------------+---------+-------------+
|                 _id|ano_ingresso|forma_ingresso| id_curso|id_unidade|id_unidade_gestora|  matricula|modalidade_educacao|nivel_ensino|          nome_curso|       nome_discente|        nome_unidade|nome_unidade_gestora|periodo_ingresso|sexo|sigla_nivel_ensino|   status|tipo_discente|
+--------------------+------------+--------------+---------+----------+------------------+-----------+-------------------+------------+--------------------+--------------------+--------------------+--------------------+----------------+----+------------------+---------+-------------+
|684f180bcace6a2b3...|        2024|          SiSU| 92127264|      6069|               605|20240004567|         PRESENCIAL|   GRADUAÇÃO|TECNOLOGIA

In [125]:
# Computar quantos alunos são do sexo masculino e do sexo feminino;
from pyspark.sql import Row

mulheres = df.filter(df.sexo == "F").count()
homens = df.filter(df.sexo == "M").count()
mulheres
homens
print(f'soma = {mulheres + homens} -> total {df.count()}')

# Cria uma lista com dicionários ou Rows
dados = [Row(sexo="F", quantidade=mulheres),
         Row(sexo="M", quantidade=homens)]

# Cria o DataFrame
df_generos = spark.createDataFrame(dados)
df_generos.show()

soma = 14458 -> total 14459


[Stage 180:>                                                        (0 + 1) / 1]

+----+----------+
|sexo|quantidade|
+----+----------+
|   F|      7470|
|   M|      6988|
+----+----------+



In [126]:
# Computar o top 5 dos cursos que mais receberam alunos;

top5 = df.groupBy('nome_curso').count().withColumnRenamed('count', 'NUMERO_ESTUDANTES').orderBy(desc('NUMERO_ESTUDANTES')).limit(5)
top5.show(truncate=False)

+--------------------------------+-----------------+
|nome_curso                      |NUMERO_ESTUDANTES|
+--------------------------------+-----------------+
|                                |1260             |
|CIÊNCIAS E TECNOLOGIA           |787              |
|CURSO DE LÍNGUA INGLESA  BÁSICO|494              |
|PEDAGOGIA                       |427              |
|TECNOLOGIA DA INFORMAÇÃO        |346              |
+--------------------------------+-----------------+



In [127]:
# Realizar consulta múltipla ("relacional"), por exemplo: quantos alunos são do sexo masculino que ingressaram via SiSU em algum curso em específico.

# estudantes que entraram pelo sisu e é de eng comp
sisu_engcomp = df.filter((df.nome_curso == "ENGENHARIA DE COMPUTAÇÃO") & (df.forma_ingresso == "SiSU"))
sisu_engcomp_count = sisu_engcomp.count()

# apenas homens
masc = sisu_engcomp.filter(df.sexo == "M")
masc_count = masc.count()

# apenas mulheres
fem = sisu_engcomp.filter(df.sexo == "F")
fem_count = fem.count()


sisu_engcomp.show()
print(f'total alunos entraram em computação pelo sisu: {sisu_engcomp_count}')
print(f'quantidade de alunos homens que entraram em computação pelo sisu: {masc_count}')
print(f'quantidade de alunos mulheres que entraram em computação pelo sisu: {fem_count}')

+--------------------+------------+--------------+--------+----------+------------------+-----------+-------------------+------------+--------------------+--------------------+--------------------+--------------------+----------------+----+------------------+--------+-------------+
|                 _id|ano_ingresso|forma_ingresso|id_curso|id_unidade|id_unidade_gestora|  matricula|modalidade_educacao|nivel_ensino|          nome_curso|       nome_discente|        nome_unidade|nome_unidade_gestora|periodo_ingresso|sexo|sigla_nivel_ensino|  status|tipo_discente|
+--------------------+------------+--------------+--------+----------+------------------+-----------+-------------------+------------+--------------------+--------------------+--------------------+--------------------+----------------+----+------------------+--------+-------------+
|684f180bcace6a2b3...|        2024|          SiSU| 2000026|       445|               445|20240020121|         PRESENCIAL|   GRADUAÇÃO|ENGENHARIA DE COM

# Salvando os dados de volta no MongoDB

In [129]:
# salvando em uma coleção os alunos do sisu
alunos_sisu.write.format("mongodb").mode("overwrite").option("connection.uri", 'mongodb://meu-mongo:27017/').option("database", "ufrn").option("collection", "alunos_sisu").save()

# salvando o top5
top5.write.format("mongodb").mode("overwrite").option("connection.uri", 'mongodb://meu-mongo:27017/').option("database", "ufrn").option("collection", "top5").save()

# salvando os alunos do sexo masculino e feminino
df_generos.write.format("mongodb").mode("overwrite").option("connection.uri", 'mongodb://meu-mongo:27017/').option("database", "ufrn").option("collection", "generos").save()

# salvando os alunos que vinheram do sisu e são de eng_comp
sisu_engcomp.write.format("mongodb").mode("overwrite").option("connection.uri", 'mongodb://meu-mongo:27017/').option("database", "ufrn").option("collection", "generos").save()

# Verificando se os dados foram realmente salvos

In [130]:
df = spark.read.format("mongodb") \
    .option("connection.uri", "mongodb://meu-mongo:27017") \
    .option("database", "ufrn") \
    .option("collection", "top5").load()

In [131]:
df.show(truncate=False)

+-----------------+------------------------+--------------------------------+
|NUMERO_ESTUDANTES|_id                     |nome_curso                      |
+-----------------+------------------------+--------------------------------+
|1260             |684f18a9809b6067682f84eb|                                |
|787              |684f18a9809b6067682f84ec|CIÊNCIAS E TECNOLOGIA           |
|494              |684f18a9809b6067682f84ed|CURSO DE LÍNGUA INGLESA  BÁSICO|
|427              |684f18a9809b6067682f84ee|PEDAGOGIA                       |
|346              |684f18a9809b6067682f84ef|TECNOLOGIA DA INFORMAÇÃO        |
+-----------------+------------------------+--------------------------------+

